<a href="https://colab.research.google.com/github/D-Yadav/NLP/blob/main/Machine_Learning_language_Translation_using_encoders_decoders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Machine Learning Language Translation**

For in-depth intution on concepts : https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html

In [1]:
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np

2.12.0


In [2]:
batch_size = 64 #batch size for training
epochs = 100 #Number of epochs to train for.
latent_dim = 256 #Latent dimensionality of the encoding space
num_samples = 10000 #No of samples to train on
data_path = '/content/fra.txt' #Path to data test file on disk

In [3]:
# Vectorize the data, we are basically reading input as well as target data, w.r.t all input sentences we are basically reading all unique characters
# Input data is english sentences and output data is all french sentences
input_texts = [] # English input data in form of texts
target_texts = [] # French o/p data in the form of texts
input_characters = set() # Gives all the unique characters of english
target_characters = set() # All the french characters in dataset
with open(data_path, 'r', encoding = 'utf-8') as f: # Open file
  lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) -1)]: # traversing to every lines
  input_text, target_text, _ = line.split('\t') # Splits w.r.t input_texts and target_texts
  # We use "tab" as the "start sequence" character
  # for the targets, and "\n" as "end-sequence" character
  target_text = '\t' + target_text + '\n' #Input Text is English texts & Target Text is french texts
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)
  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)

In [5]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [6]:
print('Number of samples :', len(input_texts))
print('Number of unique input tokens :', num_encoder_tokens)
print('Number of unique output tokens :', num_decoder_tokens)
print('Max sequence length for inputs :', max_encoder_seq_length)
print('Max sequence length for outputs :', max_decoder_seq_length)

Number of samples : 10000
Number of unique input tokens : 71
Number of unique output tokens : 93
Max sequence length for inputs : 15
Max sequence length for outputs : 59


In [8]:
# Assigning tokens to each and every characters
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate (target_characters)])

In [7]:
input_token_index, target_token_index

({' ': 0,
  '!': 1,
  '"': 2,
  '$': 3,
  '%': 4,
  '&': 5,
  "'": 6,
  ',': 7,
  '-': 8,
  '.': 9,
  '0': 10,
  '1': 11,
  '2': 12,
  '3': 13,
  '5': 14,
  '7': 15,
  '8': 16,
  '9': 17,
  ':': 18,
  '?': 19,
  'A': 20,
  'B': 21,
  'C': 22,
  'D': 23,
  'E': 24,
  'F': 25,
  'G': 26,
  'H': 27,
  'I': 28,
  'J': 29,
  'K': 30,
  'L': 31,
  'M': 32,
  'N': 33,
  'O': 34,
  'P': 35,
  'Q': 36,
  'R': 37,
  'S': 38,
  'T': 39,
  'U': 40,
  'V': 41,
  'W': 42,
  'Y': 43,
  'a': 44,
  'b': 45,
  'c': 46,
  'd': 47,
  'e': 48,
  'f': 49,
  'g': 50,
  'h': 51,
  'i': 52,
  'j': 53,
  'k': 54,
  'l': 55,
  'm': 56,
  'n': 57,
  'o': 58,
  'p': 59,
  'q': 60,
  'r': 61,
  's': 62,
  't': 63,
  'u': 64,
  'v': 65,
  'w': 66,
  'x': 67,
  'y': 68,
  'z': 69,
  'é': 70},
 {'\t': 0,
  '\n': 1,
  ' ': 2,
  '!': 3,
  '%': 4,
  '&': 5,
  "'": 6,
  '(': 7,
  ')': 8,
  ',': 9,
  '-': 10,
  '.': 11,
  '0': 12,
  '1': 13,
  '2': 14,
  '3': 15,
  '5': 16,
  '8': 17,
  '9': 18,
  ':': 19,
  '?': 20,
  'A'

In [9]:
# This step is basically to show how you can do OHE by using numpy below
encoder_input_data = np.zeros( # Initially making all zeros based on dimension
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype = 'float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype = 'float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
## Summary of the process : https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html

In [10]:
## OHE

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
  for t, char in enumerate(input_text):
    encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
      # decoder_target_data is ahead of decoder_input_data by one timestamp
      decoder_target_data[i, t, target_token_index[char]] = 1.
      if t>0:
        # decoder_target_data will be ahead by one timestamp
        # and will not include the start character.
        decoder_target_data[i, t-1, target_token_index[char]] =1.
        decoder_input_data[i, t + 1:, target_token_index[' ']] =1.
        decoder_target_data[i, t:, target_token_index[' ']] =1.

In [11]:
encoder_input_data[0].shape

(15, 71)

In [12]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True) #Initialize the LSTM Layer
encoder_outputs, state_h, state_c = encoder(encoder_inputs) #state_h is hidden cell, state_c is cell state
# We discard 'encoder_outputs' and only keep the states.
encoder_states = [state_h, state_c]

In [13]:
# Set up the decoder, using 'encoder-states' as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well, We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [15]:
# Define a model that will turn
# 'encoder_input_data' & 'decoder_input_data' into 'decoder_target_data'
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
# Compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model using fit
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 12s 21ms/step - loss: 3.7918 - accuracy: 0.9552 - val_loss: 4.4313 - val_accuracy: 0.9661
Epoch 2/100
125/125 [==============================] - 1s 11ms/step - loss: 4.1418 - accuracy: 0.9661 - val_loss: 4.9332 - val_accuracy: 0.9661
Epoch 3/100
125/125 [==============================] - 2s 12ms/step - loss: 4.5811 - accuracy: 0.9661 - val_loss: 5.4323 - val_accuracy: 0.9661
Epoch 4/100
125/125 [==============================] - 2s 13ms/step - loss: 5.0168 - accuracy: 0.9661 - val_loss: 5.9228 - val_accuracy: 0.9661
Epoch 5/100
125/125 [==============================] - 1s 11ms/step - loss: 5.4541 - accuracy: 0.9661 - val_loss: 6.4159 - val_accuracy: 0.9661
Epoch 6/100
125/125 [==============================] - 1s 12ms/step - loss: 5.8854 - accuracy: 0.9661 - val_loss: 6.9484 - val_accuracy: 0.9661
Epoch 7/100
125/125 [==============================] - 1s 11ms/step - loss: 6.3323 - accuracy: 0.9661 - val_loss: 7.4839 - val_accuracy